# POD-NN method

## The advection-diffusion problem

Let us  considere the following BVP. It is based on the 2D stationary advection-diffusion. It is here parametrized by the diffusivity coefficient $\lambda(\mu)$. The equations read: 

$$\begin{equation}
\begin{array}:
-div(\lambda(\mu)\nabla u)+w\nabla u&=f & \text{in}~~\Omega\\
u&=g & \text{in}~~\varGamma_{in}\\
-\lambda(\mu)\nabla u\cdot n&=0 & \text{in}~~\varGamma_{wall}\\
-\lambda(\mu)\nabla u\cdot n&=0 & \text{in}~~\varGamma_{out}
\end{array}
\end{equation}
$$

with $\lambda(\mu)=\exp(\mu_{0}(\mu + 1))$. The scalar $\mu_{0}=0.7$ and $\mu\in\textbf{P}=[\mu_{min},{~}\mu_{max}],{~}  \mu_{min}=1,{~} \mu_{max}=10$.

### Modules  import and initializations

In [63]:
### Modules importation
import tensorflow as tf
import tensorflow.keras as k
from tensorflow.keras.datasets import mnist
import tensorflow.keras.preprocessing.image as kpi
import tensorflow.keras.models as km
import tensorflow.keras.layers as kl
import tensorflow.keras.losses as kloss
import tensorflow.keras.regularizers as kr
import tensorflow.keras.backend as K
import tensorflow.keras.utils as ku
from tensorflow.keras import callbacks
import scipy as sc
import numpy as np
import numpy.linalg as npl
import matplotlib.pyplot as plt
import sys,os
import random 
#import pandas as pd
import sklearn.utils
# Verbosity
fit_verbosity = 1

### Normalization and non-affine functions 

In [64]:
# The scaling function for data normalisation
# Scaling function
def scaling(S, S_max, S_min):
    S[ : ] = (S - S_min)/(S_max - S_min)
    
# Inverse scaling function
def inverse_scaling(S, S_max, S_min):
    S[ : ] = (S_max - S_min) * S + S_min
    
# The non-affinity function
def Lambda(mu):
    return np.exp(mu0*(mu+1.))

## The offline phase

### Preparing the data

The RB POD matrix $Brb=[\xi_{1},\dots,\xi_{M}]\in\mathbb{R}^{NN\times N_{rb}}$. The RB matrix is constructed by POD method. It is computed for 100 snapshots of Problem (1). The reduced matrix here is of size $N_{rb}=5$.

For the training step of the NN, the snapshots matrix $S=[U_{h}(\mu_{1}),\dots,U_{h}(\mu_{M})]\in\mathbb{R}^{NN\times M}$ has already been computed with $M=10000$.

#### Load the data from numpy files

In [65]:
# Prepare the data
# Load the data from the numpy file
# Snaphots matrix
S =  np.load('Snapshots_non_affine.npy') # of size M*NN
S = S.transpose() # of size NN*M

# The reduced POD basis
Brb = np.load('Brb.npy') # of size NN*Nrb#
Brb = Brb.real
Nrb = len(Brb.transpose())

# The parameter matrix
P = np.load('parameter_non_affine.npy') # of size M x 1
P = P.reshape(len(P),1)

#### Computation of the reduced outputs for the NN and randomly shuffle the data

The reduced outputs are computed by the formula: $$\begin{equation}Urb = Brb^{T}U_{h}(\mu)\in\mathbb{R}^{N_{rb}}\end{equation}$$ with $\mu\in\textbf{P}=[\mu_{min},{~}\mu_{max}]$. $\\$
We denote by $Urb_{POD}=Brb^{T}S$.

In [66]:
# Computation of the reduced solutions: Brb^T*Uh(mu)
Urb_POD = np.dot(Brb.transpose(),S) # of size Nrb*M
#print("Urb_POD size=",Urb_POD.shape)

# Transpose 
Urb_POD = Urb_POD.transpose() # of size M*Nrb
print("Urb_POD size=",Urb_POD.shape)

# Randomy shufl the data set
shuffle = np.arange(len(Urb_POD))
np.random.shuffle(shuffle)
Urb_POD = Urb_POD[shuffle]
P = P[shuffle]
print("Urb_POD before normalization \n",Urb_POD)
print("P before normalization\n",P)

Urb_POD size= (10000, 5)
Urb_POD before normalization 
 [[-3.82874547e+01 -1.08189228e+00 -9.21283941e-02  6.70748927e-02
   1.64140633e-02]
 [-3.67402027e+01 -1.59635435e+00  3.51970723e-01 -3.38847339e-02
  -3.47409812e-03]
 [-3.97810071e+01 -4.13601831e-01 -5.23348323e-01  1.12482827e-01
   1.21719565e-02]
 ...
 [-4.04210875e+01  3.01885325e+00  1.00825938e+00  1.93867536e-01
  -2.70956354e-02]
 [-4.03359884e+01 -4.81891247e-02 -6.71382387e-01  9.53138862e-02
   8.68552809e-05]
 [-3.80288074e+01 -1.17613619e+00 -1.72229418e-02  5.27101184e-02
   1.44241288e-02]]
P before normalization
 [[170.83258954]
 [621.61021438]
 [ 75.83294685]
 ...
 [  4.62013092]
 [ 54.20153727]
 [199.97649207]]


#### Data normalization

The normalization of the data is done as follows:

The input parameters for the NN are such that: $$\begin{equation}\tilde{\mu_{i}} = \frac{\mu_{i}}{\mu_{max}}\end{equation}$$ for $1\leq i\leq M$

The outputs (RB solutions) for the NN are normalized as follows:
$$\begin{equation}
(\tilde{Urb}_{POD})_{ij} = \frac{(Urb_{POD})_{ij}-\underset{i,j}{\min}(Urb_{POD})_{ij}}{\underset{i,j}{\max}(Urb_{POD})_{ij}-\underset{i,j}{\min}(Urb_{POD})_{ij}}
\end{equation}$$ 
for $1\leq i\leq M$ and $1\leq j\leq N_{rb}$

In [67]:
# Data normalization
# Normalization of the parameter set
# Obtain the min and the max of P
P_max = np.max(P); P_min = np.min(P)
# Normalize the parameter vector P
P = P/P_max

# Normalization of the reduced matrix
# Obtain the min and the max of the reduced outputs BrbUh
Urb_POD_max = np.max(Urb_POD); Urb_POD_min = np.min(Urb_POD)
scaling(Urb_POD, Urb_POD_max, Urb_POD_min)

print("P after normalization\n",P)
print("Urb_POD after normalization\n",Urb_POD)

P after normalization
 [[0.07735764]
 [0.281482  ]
 [0.03433922]
 ...
 [0.00209212]
 [0.02454393]
 [0.09055479]]
Urb_POD after normalization
 [[0.06100667 0.90446869 0.92690696 0.93051615 0.92936765]
 [0.09608337 0.89280567 0.93697483 0.92822736 0.92891678]
 [0.02714736 0.91961905 0.91713106 0.93154556 0.92927148]
 ...
 [0.01263653 0.99743391 0.9518531  0.93339058 0.92838127]
 [0.01456576 0.92790307 0.91377508 0.93115633 0.92899751]
 [0.06687029 0.90233215 0.92860509 0.93019049 0.92932254]]


#### Load the already trained NN

In [68]:
# Load the pre-trained NN stored in h5 format
Model = km.load_model(f'Neural-network.h5')

# Summary of the model: layers and number of parameters 
Model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 17)                34        
                                                                 
 dense_1 (Dense)             (None, 17)                306       
                                                                 
 dense_2 (Dense)             (None, 17)                306       
                                                                 
 dense_3 (Dense)             (None, 17)                306       
                                                                 
 dense_4 (Dense)             (None, 17)                306       
                                                                 
 dense_5 (Dense)             (None, 17)                306       
                                                                 
 dense_6 (Dense)             (None, 17)                3

## The online phase

### New parameter value

In [69]:
# The onlinbe phase 
# Predict the new solution U_POD_NN
# The constant mu_0
mu0 = 0.7

# New value of the physical parameter mu
print('New value for mu ')
# The parameter input for the NN
#mu = np.array([[float(input())]])
mu = 5.
print("mu=",mu)

# The non-affine parameter
# TO BE COMPLETED ...
diffus = Lambda(mu)
print("diffus=",diffus)

# Normalization of the non-affine parameter
# TO BE COMPLETED ...
diffus /= P_max
print("diffus after normalization=",diffus)

New value for mu 
mu= 5.0
diffus= 66.6863310409251
diffus after normalization= 0.030197383422318497


### Compute the RB solution for the new parameter value by performing the NN

In [70]:
# Predict the reduced basis solution of the new parameter
# TO BE COMPLETED ...
Urb_pred = Model.predict(diffus.reshape(1, -1))

# Rescaling the predicted reduced basis solution
# TO BE COMPLETED ...
inverse_scaling(Urb_pred, Urb_POD_max, Urb_POD_min)

# The change of variable from the RB basis to complete FE one.
# TO BE COMPLETED ...
Uh_POD_NN = np.dot(Brb, Urb_pred.T)
Uh_POD_NN.shape

1/1 [==============================] - 0s 81ms/step


(1296, 1)

### Save the POD-NN solution in numpy file

In [71]:
np.save('Uh_POD_NN',Uh_POD_NN)